## Initial Preprocessing

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from dateutil import parser

BDM Corpus 2

Bom Dia Mercado (BDM) → xlsx file with BDM articles and more → preprocessing to CSV → export to repository → final dataset

In [2]:
# load
news_df = pd.read_excel("../data/raw/bdm-corpus-2.xlsx")

# Normalize individual DATE and TIME cells
def parse_datetime_components(date_cell, time_cell):
    try:
        # Coerce both to string and strip spaces
        date_str = str(date_cell).strip()
        time_str = str(time_cell).strip()
        
        # Combine and parse flexibly
        dt = parser.parse(f"{date_str} {time_str}", dayfirst=True)
        return dt.isoformat()
    except Exception:
        return pd.NaT  # mark invalid rows

# Create ISO 8601 Timestamp column
news_df['Timestamp'] = news_df.apply(lambda row: parse_datetime_components(row['DATE'], row['TIME']), axis=1)
news_df['Timestamp'] = pd.to_datetime(news_df['Timestamp'], errors='coerce')

# Drop old columns
news_df.drop(columns=['DATE', 'TIME', 'Index', 'DIRECTION', 'BRER', 'LABEL'], inplace=True)

# Clean newlines in ARTICLE CONTENT and COMMENTS
for col in ['HEADING', 'ARTICLE CONTENT', 'COMMENTS']:
    if col in news_df.columns:
        news_df[col] = news_df[col].astype(str).str.replace(r'[\r\n]+', ' ', regex=True).str.strip()

# Reorder columns
news_df = news_df[['Timestamp'] + [col for col in news_df.columns if col != 'Timestamp']]

# Rename "HEADING" to "Headline" "ARTICLE CONTENT" to "Article" and "COMMENTS" to "Comments"
news_df.rename(columns={
    'HEADING': 'Headline',
    'ARTICLE CONTENT': 'Article',
    'COMMENTS': 'Comments'
}, inplace=True)

# save
news_df.to_csv("../data/interim/bdm-corpus-2/stage-0.csv", index=False, encoding='utf-8-sig')

### Check for invalid rows (rows with no headlines) and drop them

In [3]:
news_df = pd.read_csv("../data/interim/bdm-corpus-2/stage-0.csv", encoding='utf-8-sig') # reload as csv to ensure correct encoding

invalid_rows = news_df[news_df['Headline'].isna()]
print(f"{len(invalid_rows)} invalid rows found in 'Headline' column.")
display(invalid_rows)

6 invalid rows found in 'Headline' column.


,Timestamp,Headline,Article,Comments
2332,2024-12-16 09:02:00,NaN,NaN,NaN
4299,2025-01-10 12:14:00,NaN,NaN,NaN
4302,2025-01-10 12:14:00,NaN,NaN,NaN
4306,2025-01-10 12:18:00,NaN,NaN,NaN
4367,2025-01-10 15:58:00,NaN,NaN,NaN
4372,2025-01-10 15:59:00,NaN,NaN,NaN


In [4]:
# drop rows with invalid headlines and resave
news_df = news_df.dropna(subset=['Headline'])
news_df.to_csv("../data/interim/bdm-corpus-2/stage-0.csv", index=False, encoding='utf-8-sig')

## Exchange Rate Preprocessing

Bloomberg → Download USD/BRL exchange rates as excel file → preprocess to CSV → export to repository → final dataset

In [5]:
# Step 0: Load 
df_usd_brl = pd.read_excel("../data/raw/usd-brl.xlsx")

# Step 1: Clean column names
df_usd_brl.columns = [col.strip() for col in df_usd_brl.columns]
df_usd_brl.rename(columns={"Date": "Raw Timestamp", "Último preço": "USD/BRL"}, inplace=True)

# Step 2: Parse "Raw Timestamp" directly into pandas datetime (no ISO string conversion)
df_usd_brl["Timestamp"] = pd.to_datetime(df_usd_brl["Raw Timestamp"], errors="coerce")

# Step 3: Drop the original column
df_usd_brl.drop(columns=["Raw Timestamp"], inplace=True)

# Step 4: Reorder columns
df_usd_brl = df_usd_brl[["Timestamp", "USD/BRL"]]

# Step 5: Save
df_usd_brl.to_csv("../data/interim/usd-brl.csv", index=False, encoding="utf-8-sig")

Clean up the interim stage of the exchange rate file

In [6]:
import pandas as pd
df = pd.read_csv("../data/interim/usd-brl.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values('Timestamp').reset_index(drop=True) # Sort to ensure chronological order

# Check for duplicate timestamps
duplicate_rows = df[df.duplicated(subset=['Timestamp'], keep=False)]
if not duplicate_rows.empty:
    print("Duplicate timestamps found:")
    print(duplicate_rows)
else:
    print("No duplicate timestamps found.")

# Remove duplicates (keeping first occurrence)
df = df.drop_duplicates(subset=['Timestamp'], keep='first').reset_index(drop=True)

Duplicate timestamps found:
                Timestamp  USD/BRL
1617  2024-09-05 09:00:00   5.6409
1618  2024-09-05 09:00:00   5.6409
1619  2024-09-05 09:01:00   5.6437
1620  2024-09-05 09:01:00   5.6437
1621  2024-09-05 09:02:00   5.6413
...                   ...      ...
35023 2024-11-14 17:56:00   5.7945
35024 2024-11-14 17:57:00   5.7958
35025 2024-11-14 17:57:00   5.7958
35026 2024-11-14 17:58:00   5.7964
35027 2024-11-14 17:58:00   5.7964

[11858 rows x 2 columns]


In [7]:
# Detect intraday gaps
df['TimeDiff'] = df['Timestamp'].diff()

intraday_gaps = df[
    (df['TimeDiff'] > pd.Timedelta(minutes=1)) &
    (df['Timestamp'].dt.date == df['Timestamp'].shift().dt.date)
]

# Group gaps by date
intraday_gap_summary = {}
for idx, row in intraday_gaps.iterrows():
    prev_time = df.loc[idx - 1, 'Timestamp']
    curr_time = row['Timestamp']
    gap_minutes = int(row['TimeDiff'].total_seconds() // 60)
    date = curr_time.date()
    intraday_gap_summary.setdefault(date, []).append(
        (prev_time.time(), curr_time.time(), gap_minutes)
    )

# Display results
if intraday_gap_summary:
    print("\nIntraday gaps found:")
    for date, gaps in intraday_gap_summary.items():
        print(f"\nDate: {date}")
        for prev_t, curr_t, gap in gaps:
            print(f"  {prev_t} → {curr_t}  ({gap} min gap)")
else:
    print("\nNo intraday gaps found.")

# Remove days with intraday gaps
if intraday_gap_summary:
    gap_dates = set(intraday_gap_summary.keys())
    df = df[~df['Timestamp'].dt.date.isin(gap_dates)].reset_index(drop=True)
    print(f"\nRemoved all rows from days with intraday gaps: {gap_dates}")

# Drop helper column
df = df.drop(columns=['TimeDiff'])


Intraday gaps found:

Date: 2024-12-24
  09:14:00 → 09:16:00  (2 min gap)
  09:16:00 → 09:23:00  (7 min gap)
  09:23:00 → 09:27:00  (4 min gap)
  09:28:00 → 09:53:00  (25 min gap)
  09:54:00 → 09:57:00  (3 min gap)
  09:57:00 → 09:59:00  (2 min gap)
  09:59:00 → 10:02:00  (3 min gap)
  10:03:00 → 10:06:00  (3 min gap)
  10:07:00 → 10:32:00  (25 min gap)

Removed all rows from days with intraday gaps: {datetime.date(2024, 12, 24)}


In [8]:
df.to_csv("../data/processed/usd-brl.csv", index=False)

## Stage 1
  - remove all rows following the last timestamp in the interim/bdm-corpus-2/stage-0.csv at 2024-12-30 17:32:00
  - remove article and comments columns from interim/bdm-corpus-2/stage-0.csv
  - merge the USD/BRL values from interim/usd-brl-continuous.csv into interim/bdm-corpus-2/stage-0.csv by matching timestamps
  - compute forward returns based on the price from t+1 to t+20 minutes subtracted from the price during the timestamp of the news
    - positive returns will map to +1, negative returns will map to -1, and no change will map to 0
    - each computed forward return will be stored in a new column named "Forward Return t+X" where X is the number of minutes ahead

    - incorporate a stability threshold when mapping "no change":
      - unchanged is defined as an absolute log return less than or equal to the Nth percentile of all 1-minute absolute log returns in the dataset (default: 60th percentile)
      - this replaces the exact-zero check for unchanged, allowing small price movements to be considered stable


In [10]:
# INCLUDE stable metric

import pandas as pd
import numpy as np

# Load datasets
fx_df = pd.read_csv("../data/processed/usd-brl.csv", parse_dates=["Timestamp"])
news_df = pd.read_csv("../data/interim/bdm-corpus-2/stage-0.csv", parse_dates=["Timestamp"])

# Restrict news to timestamps before cutoff
last_timestamp = pd.to_datetime("2024-12-30 17:58:00")
news_df = news_df[news_df["Timestamp"] < last_timestamp]
news_df = news_df[["Timestamp", "Headline"]]

'''
Compute forward returns with day-end invalidation
Remove 2nd parameter and argue 3rd param to False for no band
'''
def compute_forward_returns(df, horizon_minutes=20, stable_quantile=0.60, use_stable_band=True):
    df = df.copy()

    if use_stable_band:
        r1 = np.log(df["USD/BRL"]).diff().abs()
        thr = r1.quantile(stable_quantile)
    else:
        thr = 0  # unchanged means exactly zero change

    df["Date"] = df["Timestamp"].dt.date
    day_end = df.groupby("Date")["Timestamp"].transform("max")
    for i in range(1, horizon_minutes + 1):
        col = f"Forward Return t+{i}"
        ret_i = np.log(df["USD/BRL"].shift(-i)) - np.log(df["USD/BRL"])
        s = np.sign(ret_i)
        s = np.where(ret_i.abs() <= thr, 0, s)
        s = pd.Series(s, index=df.index).where(df["Timestamp"] + pd.Timedelta(minutes=i) <= day_end, np.nan)
        df[col] = s
    return df.drop(columns="Date")

fx_df = compute_forward_returns(fx_df, stable_quantile=0.60, use_stable_band=True)

# Merge into news_df
merged_df = pd.merge(news_df, fx_df, on="Timestamp", how="left")

# Key checks
print(f"Total news rows before cutoff: {len(news_df)}")
print(f"News timestamps matching FX bars: {merged_df['USD/BRL'].notna().sum()}")
for i in (1, 5, 10, 20):
    col = f"Forward Return t+{i}"
    invalid_count = merged_df[col].isna().sum()
    print(f"Invalid {col} (NaN): {invalid_count}")

Total news rows before cutoff: 3523
News timestamps matching FX bars: 3506
Invalid Forward Return t+1 (NaN): 20
Invalid Forward Return t+5 (NaN): 25
Invalid Forward Return t+10 (NaN): 35
Invalid Forward Return t+20 (NaN): 108


In [11]:
merged_df.to_csv(
    "../data/interim/bdm-corpus-2/stage-1.csv",
    index=False,
    encoding="utf-8-sig",
    na_rep="NA" # will be used later to identify which forward return horizons we can't use when assigning ground truth
)

## Stage 2
- assign t+5 minutes forward return as ground truth (Direção) - map from +1/-1 to increase/decrease
- remove all t+X forward returns EXCEPT t+5
- remove usd/brl column, as it's not needed for this since the ground truth delta is already calculated

Intraday Trading Sessions: Remove all non-intraday data, includes any news outside of market hours (weekends/holidays)
- Data now contains news from 5 minutes before market open till 5 minutes before market close
- For news at end of trading day, the price change is determined by the last trading price (not always t+5), very few instances of this

Binary Classification Task: Remove any rows that contain no change in price **(forward return == 0)** beteen timstamp t and t+5.
- 54 headlines (rows) were removed as they were either neutral (news on weekends also removed)
- Better as a binary classification task and let "hold asset" be determined by the trader based on profits or num of trades

In [12]:
import pandas as pd
merged_df = pd.read_csv("../data/interim/bdm-corpus-2/stage-1.csv", na_values=["NA"])

def make_horizon_df(df, horizon):
    """
    Create a filtered DataFrame for a specific forward return horizon.
    Keeps only Timestamp, Headline, and mapped Direction.
    Reusable function
    """
    col_name = f"Forward Return t+{horizon}"
    if col_name not in df.columns:
        raise ValueError(f"Column '{col_name}' not found in DataFrame.")

    out = df[["Timestamp", "Headline", col_name]].copy()
    out.rename(columns={col_name: "Direction"}, inplace=True)

    # Map numeric to labels
    mapping = {1.0: "Increase", -1.0: "Decrease", 0.0: "Stable"}
    out["Direction"] = out["Direction"].map(mapping)

    # Drop non-intraday / invalid forward return rows
    out = out.dropna(subset=["Direction"])

    return out

# Usage (t+5 for now)
t5_df = make_horizon_df(merged_df, 5)
t5_df.to_csv("../data/interim/bdm-corpus-2/stage-2.csv", index=False, encoding="utf-8-sig")


## Stage 3
Preparing for language model inference
- DO NOT remove stopwords
- DO NOT lemmatize or stem
- DO NOT lowercase
- DO NOT translate or normalize to English
- Preserve accents, diacritics, and original formatting


In [13]:
df = pd.read_csv("../data/interim/bdm-corpus-2/stage-2.csv")

In [14]:
# === Define robust headline cleaner ===
def clean_headline(text: str) -> str:
    text = str(text).strip()

    # Normalize smart quotes and apostrophes
    text = re.sub(r"[“”]", '"', text)
    text = re.sub(r"[‘’]", "'", text)

    # Remove repeated quotes/apostrophes
    text = re.sub(r'"{2,}', '"', text)
    text = re.sub(r"'{2,}", "'", text)

    # Remove leading/trailing quotes (even if multiple)
    text = re.sub(r'^(["\']+)', '', text)
    text = re.sub(r'(["\']+)$', '', text)

    # Remove noisy special character sequences
    text = re.sub(r"[_•√×+÷=<>^~|#*@¬]{2,}", " ", text)
    text = re.sub(r"[_•√×+÷=<>^~|#*@¬]", "", text)

    # Normalize unicode
    text = unicodedata.normalize("NFKC", text)

    # Remove non-printable/unusual characters except Latin-1
    text = re.sub(r"[^\x20-\x7EÀ-ÿ°€¢£¥‰–—…]", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)

    return text.strip()

In [15]:
df["Headline"] = df["Headline"].apply(clean_headline)
df.to_csv("../data/interim/bdm-corpus-2/stage-3.csv", index=False)

## Stage 4
- remove duplicate rows (eg., row_a timestamp/headline == row_b timestamp/headline)
    - note: some timestamps contain multiple headlines. They are all different, no need to fret over this
- removed all rows with duplicate "Manchete" column value and kept first occurence
- chronologically create train/test set for temporal evaluation
- keep test set slightly imbalanced, as it already is, to mimic real world scenario
- 3465 headlines -> 3457 headlines

In [16]:
import pandas as pd
from babel.dates import format_datetime

df = pd.read_csv("../data/interim/bdm-corpus-2/stage-3.csv")
print("Total rows before removing duplicates:", len(df))

# Drop exact (Timestamp, Headline) duplicates
dupes = df[df.duplicated(subset=["Timestamp", "Headline"], keep=False)]
print("Duplicate rows:\n", dupes)
df = df.drop_duplicates(subset=["Timestamp", "Headline"], keep="first")

# Drop headline duplicates (model shouldn't be trained on multiple instances of a headline, for semantic reasons)
dupe_heads = df[df.duplicated(subset="Headline", keep=False)]
print("Duplicate Headline count:", len(dupe_heads))
df = df.drop_duplicates(subset="Headline", keep="first")

print(f"\nTotal rows after removing duplicates: {len(df)}")

Total rows before removing duplicates: 3498
Duplicate rows:
                 Timestamp                                           Headline  \
238   2024-11-26 10:02:00                                  Reação ao IPCA-15   
241   2024-11-26 10:02:00                                  Reação ao IPCA-15   
488   2024-11-28 09:02:00  AOVIVO/Haddad sobre IR: Nosso objetivo é que e...   
489   2024-11-28 09:02:00  Ela irá beneficiar todo mundo que ganha até 5 ...   
490   2024-11-28 09:02:00  Com essa fórmula de cálculo, a suposta renúnci...   
493   2024-11-28 09:02:00  AOVIVO/Haddad sobre IR: Nosso objetivo é que e...   
494   2024-11-28 09:02:00  Ela irá beneficiar todo mundo que ganha até 5 ...   
495   2024-11-28 09:02:00  Com essa fórmula de cálculo, a suposta renúnci...   
599   2024-11-28 11:54:00  MERCADOS: Sob pressão do fiscal, Ibovespa amea...   
600   2024-11-28 11:54:00  MERCADOS: Sob pressão do fiscal, Ibovespa amea...   
851   2024-12-02 10:56:00  AOVIVO/Galípolo diz que a questã

In [ ]:
df.to_csv("../data/interim/bdm-corpus-2/stage-4.csv", index=False)

## Stage 5 (Temporary) - filtering out noisy headlines
REMOVED:
- Removed via code:
    - Anything that starts with "Reação" since lacks context and articles are too noisy
- Manually removed: (check row numbers in stage-4), re-do everytime when running preprocessing from beginning... should change that:
    - Row 3129: 2024-12-26 13:29:00, "Inicialmente, dado seria publicado à tarde"
    - Row 574: 2024-11-28 11:25:00, "Mensagem apagada"
    - Row 587: 2024-11-28 12:44:00, "Mídia oculta"

Important terms:
- losses: perda, queda, recuo
- fiscal, abaixo (fell under), Galípolo (prez of central bank of BR)

Terms to Ignore: Ibovespa, Stoxx600, Dow, S&P, Nasdaq (stock market terms)

Change labels to: em alta/baixa or valorização/desvalorização

Create with datasets with neutral and without (filtered) for binary classification

In [20]:
import pandas as pd
df = pd.read_csv("../data/interim/bdm-corpus-2/stage-4.csv")

removed = df[df['Headline'].str.startswith("Reação", na=False)]
df = df[~df['Headline'].str.startswith("Reação", na=False)]
print(removed)

df.to_csv("../data/processed/bdm-corpus-2/stage-5/data.csv", index=False)

                Timestamp                                           Headline  \
46    2024-11-22 09:00:00                    Reação aos PMIs da Zona do euro   
159   2024-11-25 12:36:00                      Reação a Conta Corrente e IDP   
231   2024-11-26 10:00:00                                  Reação ao IPCA-15   
444   2024-11-27 16:07:00                                    Reação ao Caged   
537   2024-11-28 10:44:00                            Reação ao pacote fiscal   
543   2024-11-28 10:57:00                    Reação ao pacote/Bradesco Asset   
577   2024-11-28 12:08:00                                   Reação ao pacote   
641   2024-11-29 10:01:00                                      Reação à Pnad   
745   2024-11-29 16:12:00                            Reação à bandeira verde   
785   2024-12-02 10:25:00                      Reação ao PMI da zona do euro   
894   2024-12-03 09:37:00                             Reação ao PIB do 3oTri   
904   2024-12-03 09:47:00               

In [2]:
import pandas as pd
df = pd.read_csv("../data/processed/bdm-corpus-2/stage-5/data.csv")

# Original
train_original = df.iloc[:-500]
test_original = df.tail(500)

# Filtered (remove stable first, for equal sets)
df_filtered = df[df['Direction'] != 'Stable']
train_filtered = df_filtered.iloc[:-500]
test_filtered = df_filtered.tail(500)

# Save
train_original.to_csv("../data/processed/bdm-corpus-2/stage-5/train_original.csv", index=False)
test_original.to_csv("../data/processed/bdm-corpus-2/stage-5/test_original.csv", index=False)
train_filtered.to_csv("../data/processed/bdm-corpus-2/stage-5/train_filtered.csv", index=False)
test_filtered.to_csv("../data/processed/bdm-corpus-2/stage-5/test_filtered.csv", index=False)

In [3]:
import pandas as pd
from pathlib import Path

base_path = Path("../data/processed/bdm-corpus-2/stage-5")

datasets = {
    "Train (Original)": pd.read_csv(base_path / "train_original.csv"),
    "Test (Original)": pd.read_csv(base_path / "test_original.csv"),
    "Train (Filtered)": pd.read_csv(base_path / "train_filtered.csv"),
    "Test (Filtered)": pd.read_csv(base_path / "test_filtered.csv")
}

for name, df in datasets.items():
    print(f"\n{name}")
    print(df['Direction'].value_counts())



Train (Original)
Direction
Increase    1096
Decrease    1046
Stable       676
Name: count, dtype: int64

Test (Original)
Direction
Increase    214
Decrease    155
Stable      131
Name: count, dtype: int64

Train (Filtered)
Direction
Increase    1045
Decrease     966
Name: count, dtype: int64

Test (Filtered)
Direction
Increase    265
Decrease    235
Name: count, dtype: int64
